In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
w_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=w_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Vila Velha,BR,2022-07-05 19:26:37,-20.3297,-40.2925,78.75,78,20,14.97
9,9,Albany,US,2022-07-05 19:24:32,42.6001,-73.9662,76.39,70,100,8.95
10,10,Saint Anthony,US,2022-07-05 19:26:40,45.0205,-93.2180,89.15,67,0,9.22
12,12,Laguna,US,2022-07-05 19:26:40,38.4210,-121.4238,87.44,52,0,9.22
13,13,Poum,NC,2022-07-05 19:26:41,-20.2333,164.0167,76.66,81,100,23.13
14,14,Salalah,OM,2022-07-05 19:26:41,17.0151,54.0924,80.69,89,100,5.75
16,16,Carauari,BR,2022-07-05 19:26:42,-4.8828,-66.8958,86.92,59,68,1.81
22,22,Rikitea,PF,2022-07-05 19:26:46,-23.1203,-134.9692,76.05,77,2,17.72
24,24,Imbituba,BR,2022-07-05 19:22:57,-28.2400,-48.6703,75.85,76,100,4.14
26,26,Muisne,EC,2022-07-05 19:26:47,0.6000,-80.0333,76.59,80,83,9.44


In [17]:
# Look for null values
preferred_cities_df.count()

City_ID       178
City          178
Country       178
Date          178
Lat           178
Lng           178
Max Temp      178
Humidity      178
Cloudiness    178
Wind Speed    178
dtype: int64

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Vila Velha,BR,78.75,-20.3297,-40.2925,
9,Albany,US,76.39,42.6001,-73.9662,
10,Saint Anthony,US,89.15,45.0205,-93.2180,
12,Laguna,US,87.44,38.4210,-121.4238,
13,Poum,NC,76.66,-20.2333,164.0167,
14,Salalah,OM,80.69,17.0151,54.0924,
16,Carauari,BR,86.92,-4.8828,-66.8958,
22,Rikitea,PF,76.05,-23.1203,-134.9692,
24,Imbituba,BR,75.85,-28.2400,-48.6703,
26,Muisne,EC,76.59,0.6000,-80.0333,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Vila Velha,BR,78.75,-20.3297,-40.2925,Hotel Vitória Palace
9,Albany,US,76.39,42.6001,-73.9662,
10,Saint Anthony,US,89.15,45.0205,-93.2180,Holiday Inn Express Roseville-St. Paul
12,Laguna,US,87.44,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
13,Poum,NC,76.66,-20.2333,164.0167,
14,Salalah,OM,80.69,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
16,Carauari,BR,86.92,-4.8828,-66.8958,Hotel Fellip
22,Rikitea,PF,76.05,-23.1203,-134.9692,People ThankYou
24,Imbituba,BR,75.85,-28.2400,-48.6703,Praia Hotel Imbituba
26,Muisne,EC,76.59,0.6000,-80.0333,Quinta Samay


In [28]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Example of info_box_template HTML code - DO NOT NEED TO RUN - for example only

#            info_box_template = """
#            <dl>
#            <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
#            <dt>City</dt><dd>{City}</dd>
#            <dt>Country</dt><dd>{Country}</dd>
#            <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
#            </dl>
#            """

In [38]:
# Store the DataFrame Row. - Stores HTML code for each row in the hotel_df data frame.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [39]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))